In [1]:
import numpy as np
from dppy.finite_dpps import FiniteDPP

In [2]:
correlations = np.eye(5)
dpp = FiniteDPP('correlation', **{'K': correlations})



In [3]:
dpp.sample_exact()
dpp.list_of_samples

[array([0, 3, 2, 1, 4])]